# 03 — References, Pointers & The Cost of Copying 📉

## 1. The Copy Trap: `struct A a = b;`

This is the most important behavioral difference between C and C++ assignments.

### The C Scenario (Shallow Copy)
In C, `a = b` performs a bitwise copy (`memcpy`). 
* If the struct contains a pointer, only the address is copied.
* Result: Two structs pointing to the same buffer. Double-free crashes are common.

### The C++ Scenario (Deep Copy)
In C++, containers like `std::vector` and `std::string` define a Copy Constructor.
* `a = b` allocates NEW heap memory for `a` and copies the DATA from `b`.
* **Benefit:** Safety. `a` and `b` are independent.
* **Cost:** Performance. This is an O(N) malloc/copy operation.


In [1]:
#include <vector>
#include <iostream>

{
    std::vector<int> original = {1, 2, 3, 4, 5};

    // In C, this would be a pointer copy.
    // In C++, this ALLOCATES new memory and copies all elements.
    std::vector<int> copy = original;

    // Modify the copy
    copy[0] = 100;

    // Check the original
    std::cout << "Copy[0]: " << copy[0] << std::endl;
    std::cout << "Original[0]: " << original[0] << " (Unchanged!)" << std::endl;

    // Proof: Addresses are different
    std::cout << "Original Data @ " << original.data() << std::endl;
    std::cout << "Copy Data     @ " << copy.data() << std::endl;
}

Copy[0]: 100
Original[0]: 1 (Unchanged!)
Original Data @ 0x5d220d7c2c20
Copy Data     @ 0x5d220d81ba60


## 2. The Solution: References (`&`)

Since copying is expensive, we need a way to pass objects around cheaply. 
In C, you use pointers: `void func(const int* ptr)`.
In C++, we prefer References: `void func(const int& ref)`.

### References vs Pointers

| Feature | Pointer (`T*`) | Reference (`T&`) |
| :--- | :--- | :--- |
| **Null?** | Can be `nullptr` | **Must** exist |
| **Reseat?** | Can point to new target | Bound at birth |
| **Syntax** | Explicit `*` and `->` | Transparent (looks like value) |


In [ ]:
#include <string>

// BAD: Passes by Value (Triggers a COPY)
void bad_print(std::string s) {
    std::cout << "Bad: " << s << std::endl;
}

// GOOD: Passes by Reference (No Copy)
// const: "I promise not to change it"
// &: "Pass the memory address, not the value"
void good_print(const std::string& s) {
    std::cout << "Good: " << s << std::endl;
}

{
    std::string huge_text = "Imagine this text is 100MB big...";

    // bad_print(huge_text); // Allocates 100MB, copies, prints, frees.
    good_print(huge_text);   // Passes 8 bytes (pointer), prints.
}

## 3. Summary

1. **Assignment (`=`) is Deep Copy.** It is safe but slow for large objects.
2. **Use `const T&`** for function arguments when you want to read data without copying.
3. **Use `T&`** when you want to modify data in place.
4. **Use `T*`** only when "no object" (`nullptr`) is a valid state.